In [8]:
import pandas
import os.path
from glob import glob
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LogNorm
from matplotlib.patches import Patch
from matplotlib.lines import Line2D




In [31]:
#Uvozimo tabele
paths = glob("tables/*.csv")
league_paths = [paths[0:4], paths[4:9], paths[9:13], paths[13:17], paths[17:]]
leagues = ["bundesliga", "laliga", "ligue 1", "premier league", "serie a"]

#Iz tabel izluščimo prednost prvouvščenega in njegovo povprečje točk na tekmo
result = pandas.DataFrame({"League": [], "PTS Difference" : [], "Points Per Game" : []})
for league, name in zip(league_paths, leagues):
    for el in league:
        tabela = pandas.read_csv(el, index_col = 0)
        pts_diff = tabela.iloc[0]["PTS"] - tabela.iloc[1]["PTS"]
        ppg = tabela.iloc[0]["PTS"]/(tabela.iloc[0]["Wins"] + tabela.iloc[0]["Losses"] + tabela.iloc[0]["Draws"])
        new_row = {"League" : name, "PTS Difference" : pts_diff, "Points Per Game" : ppg}
        result = result.append(new_row, ignore_index=True)
result

,League,PTS Difference,Points Per Game
0,bundesliga,6.0,2.400000
1,bundesliga,21.0,2.470588
2,bundesliga,2.0,2.294118
3,bundesliga,13.0,2.411765
4,laliga,1.0,2.394737
5,laliga,3.0,2.447368
6,laliga,14.0,2.447368
7,laliga,11.0,2.289474
8,laliga,5.0,2.289474
9,ligue 1,31.0,2.526316


In [32]:
paths = glob("data/*.csv")
league_paths = [paths[0:4], paths[4:9], paths[9:13], paths[13:17], paths[17:]]
leagues = ["bundesliga", "laliga", "ligue 1", "premier league", "serie a"]

result = pandas.DataFrame({"League": [], "Goals per Game" : []})
for league, name in zip(league_paths, leagues):
    for el in league:
        tabela = pandas.read_csv(el, index_col = 0)
        gpg = tabela["goals"].mean() + tabela["opponent goals"].mean()
        new_row = {"League" : name, "Goals per Game" : gpg}
        result = result.append(new_row, ignore_index=True)
result


,League,Goals per Game
0,bundesliga,3.096296
1,bundesliga,2.794118
2,bundesliga,3.179739
3,bundesliga,3.209150
4,laliga,2.744737
5,laliga,2.942105
6,laliga,2.694737
7,laliga,2.586842
8,laliga,2.478947
9,ligue 1,2.526316
